<h1>Primeiro Web Scraping</h1>

Primeiro projeto de web scraping utilizando dados reais.
Será utilizada uma página do SUS para estudo da biblioteca <font color='blue'>BeautifulSoup4</font> na coleta de dados em páginas web.

Página: http://tabnet.datasus.gov.br/cgi/idb2012/matriz.htm

É necessário utilizar a biblioteca <font color='blue'>urllib</font> que possui a função que realiza uma requisiçã GET no servidor WEB através de uma url passada.

In [1]:
from urllib.request import urlopen

html = urlopen('http://tabnet.datasus.gov.br/cgi/idb2012/matriz.htm')
print(html.read())

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n\r\n<html lang="pt-BR">\r\n\r\n<head>\r\n   <title>Indicadores e Dados B\xe1sicos - Brasil - 2012</title>\r\n   <meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">\r\n   <meta name="GENERATOR" content="DATASUS">\r\n   <link rel="stylesheet" href="/tabnet/idb.css" type="text/css">\r\n   <link rel="stylesheet" href="/tabnet/idbp.css" media="print" type="text/css">\r\n   <script type="text/javascript">\r\n      function jumpMenu(selObj){\r\n         eval("self.location=\'"+selObj.options[selObj.selectedIndex].value+"\'");\r\n         selObj.selectedIndex=0;\r\n         }\r\n      </script>\r\n</head>\r\n\r\n<body>\r\n\r\n<a name="topo"></a>\r\n\r\n<div name="governo" class="todalargura">\r\n\r\n<table class="testeira" cellspacing="0" cellpadding="0">\r\n\r\n   <tr valign="top" class="Escondido" style="background-color: #ffcc00;">\r\n      <td align="left" width=

Com a biblioteca padrão do python é possível obter todo o conteúdo html da página requisitada.
O <font color='blue'>BeatifulSoup</font> entra com funções que facilitam a manipulação deste conteúdo.

In [4]:
from bs4 import BeautifulSoup

html = urlopen('http://tabnet.datasus.gov.br/cgi/idb2012/matriz.htm')
bs = BeautifulSoup(html.read(), 'html.parser')

print(bs)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="pt-BR">
<head>
<title>Indicadores e Dados Básicos - Brasil - 2012</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="DATASUS" name="GENERATOR"/>
<link href="/tabnet/idb.css" rel="stylesheet" type="text/css"/>
<link href="/tabnet/idbp.css" media="print" rel="stylesheet" type="text/css"/>
<script type="text/javascript">
      function jumpMenu(selObj){
         eval("self.location='"+selObj.options[selObj.selectedIndex].value+"'");
         selObj.selectedIndex=0;
         }
      </script>
</head>
<body>
<a name="topo"></a>
<div class="todalargura" name="governo">
<table cellpadding="0" cellspacing="0" class="testeira">
<tr class="Escondido" style="background-color: #ffcc00;" valign="top">
<td align="left" width="50%">
<a href="http://www.saude.gov.br" target="_blank">
<img alt="Ministério da Saúde" height="21" id="img1" src="http:/

Uma primeira vantagem observada é a organização do conteúdo da página, incluindo a identação para visualização utilizando a função <font color='blue'>print()</font>.

<h4>Tratando Excessões</h4>

Dois erros podem ocorrer durante a chamada da função <font color='blue'>urlopen()</font>:
<ol>
    <li>A página não é encontrada no servidor, ou houve um erro ao obtê-la.</li>
    <li>O servidor não foi encontrado.</li>
</ol>

No primeiro caso, a função irá lançar uma exceção genérica <font color='red'>HTTPError</font>.

Se o servidor não for encontrado ou houver um erro na url, a função <font color='blue'>urlopen()</font> irá lançar uma exceção <font color='red'>URLError</font>. Isso significa que não foi possível acessar o servidor, logo como o servidor é responsável por retornar os códigos de status HTTP, não é possível lançar um <font color='red'>HTTPError</font>.

É necessário capturar e tratar ambas as exceções.


In [8]:
from urllib.error import HTTPError
from urllib.error import URLError

try:
    html = urlopen('http://tabnet.datasus.gov.br/cgi/idb2012/matriz.htm')
except HTTPError as e:
    print(e)
except URLError as e:
    print('O servidor não pode ser encontrado.')
else:
    bs = BeautifulSoup(html.read(), 'html.parser')
    print(bs)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="pt-BR">
<head>
<title>Indicadores e Dados Básicos - Brasil - 2012</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="DATASUS" name="GENERATOR"/>
<link href="/tabnet/idb.css" rel="stylesheet" type="text/css"/>
<link href="/tabnet/idbp.css" media="print" rel="stylesheet" type="text/css"/>
<script type="text/javascript">
      function jumpMenu(selObj){
         eval("self.location='"+selObj.options[selObj.selectedIndex].value+"'");
         selObj.selectedIndex=0;
         }
      </script>
</head>
<body>
<a name="topo"></a>
<div class="todalargura" name="governo">
<table cellpadding="0" cellspacing="0" class="testeira">
<tr class="Escondido" style="background-color: #ffcc00;" valign="top">
<td align="left" width="50%">
<a href="http://www.saude.gov.br" target="_blank">
<img alt="Ministério da Saúde" height="21" id="img1" src="http:/

<h4>Utilizando o BeautifulSoup</h4>  

O objeto beautifulsoup retornado organiza os elementos em uma estrutura de árvore, permitindo que elementos sejam localizados através de suas tags html.

Inicialmente, vamos buscar todos os elementos de class="Nivel1L", e todos de class="Nivel2".
Os elementos class="Nivel1L" possuem tag <font color='red'>a</font>.
Os elementos class="Nivel2" possuem tag <font color='red'>ul</font>.

In [16]:
#encapsulando o código para obter conteúdo da página web em uma função:
def getHtmlContent(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        print(e)
    except URLError as e:
        print('O servidor não pode ser encontrado.')
    else:
        bs = BeautifulSoup(html.read(), 'html.parser')
        return bs
        
url = 'http://tabnet.datasus.gov.br/cgi/idb2012/matriz.htm'
bs = getHtmlContent(url)
n1list = bs.findAll('a',{'class':'Nivel1L'})
print("======================================================================================")
print("Elementos de classe Nivel1L:")
print("======================================================================================")
for name in n1list:
    print(name.get_text())
n2list = bs.findAll('ul', {'class':'Nivel2'})
print("======================================================================================")
print("Elementos de classe Nivel2:")
print("======================================================================================")
for name in n2list:
    print(name.get_text())

Elementos de classe Nivel1L:
Apresentação
O que há de novo
Indicadores demográficos
Indicadores socioeconômicos
Indicadores de mortalidade
Indicadores de morbidade
Indicadores de fatores de risco e proteção
Indicadores de recursos
Indicadores de cobertura
Folheto do IDB-2012
Publicação "Indicadores básicos para a saúde no Brasil:
       conceitos e aplicações"
Elementos de classe Nivel2:

População total - A.1
Razão de sexo - A.2
Taxa de crescimento da população - A.3
Grau de urbanização - A.4
Proporção de menores de 5 anos de idade na população - A.13
Proporção de idosos na população - A.14
Índice de envelhecimento - A.15
Razão de dependência - A.16
Razão entre nascidos vivos informados e estimados - A.17
Taxa bruta de natalidade - A.7
Taxa específica de fecundidade - A.6
Taxa de fecundidade total - A.5
Razão entre óbitos informados e estimados - A.18
Mortalidade proporcional por idade - A.8
Mortalidade proporcional por idade, em menores de 1 ano de idade - A.9
Taxa bruta de mortalida

Vamos testar a função <font color='blue'>find()</font> do <font color='blue'>BeautifulSoup</font> para retornar o conteúdo da primeira <font color='blue'>div</font> com <font color='blue'>class="Nivel1":

In [17]:
print(bs.find('div', {'class':'Nivel1'}))

<div class="Nivel1">
<h1 class="Nivel0"><a name="demog"></a>A. Indicadores demográficos</h1>
<ul class="Nivel2">
<li><a href="/cgi/deftohtm.exe?idb2012/a01.def">População total - A.1</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a02.def">Razão de sexo - A.2</a>
<li><a href="a03.htm">Taxa de crescimento da população - A.3</a>
<li><a href="a04.htm">Grau de urbanização - A.4</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a13.def">Proporção de menores de 5 anos de idade na população - A.13</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a14.def">Proporção de idosos na população - A.14</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a15.def">Índice de envelhecimento - A.15</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a16.def">Razão de dependência - A.16</a>
<li><a href="a17.htm">Razão entre nascidos vivos informados e estimados - A.17</a>
<li><a href="a07.htm">Taxa bruta de natalidade - A.7</a>
<li><a href="a06.htm">Taxa específica de fecundidade - A.6</a>
<li><a href="a05.htm">Taxa de fecundidade total - A.5

Observa-se que ela contém todo o conteúdo com cabeçalhos e listas, de classes Nivel0 e Nivel2.
A partir dessa estrutura é possível caminhar sobre os elementos aninhados através de suas tags. Como no exemplo abaixo:

In [18]:
div1 = bs.find('div',{'class':'Nivel1'})
print(div1.ul.li)

<li><a href="/cgi/deftohtm.exe?idb2012/a01.def">População total - A.1</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a02.def">Razão de sexo - A.2</a>
<li><a href="a03.htm">Taxa de crescimento da população - A.3</a>
<li><a href="a04.htm">Grau de urbanização - A.4</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a13.def">Proporção de menores de 5 anos de idade na população - A.13</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a14.def">Proporção de idosos na população - A.14</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a15.def">Índice de envelhecimento - A.15</a>
<li><a href="/cgi/deftohtm.exe?idb2012/a16.def">Razão de dependência - A.16</a>
<li><a href="a17.htm">Razão entre nascidos vivos informados e estimados - A.17</a>
<li><a href="a07.htm">Taxa bruta de natalidade - A.7</a>
<li><a href="a06.htm">Taxa específica de fecundidade - A.6</a>
<li><a href="a05.htm">Taxa de fecundidade total - A.5</a>
<li><a href="a18.htm">Razão entre óbitos informados e estimados - A.18</a>
<li><a href="/cgi/deftohtm.exe?id

Observação: A função <font color='blue'>get_text()</font> remove as tags html do objeto e permite acessar apenas o conteúdo textual disponibilizado para o usuário na página web.

In [19]:
print(div1.ul.li.get_text())

População total - A.1
Razão de sexo - A.2
Taxa de crescimento da população - A.3
Grau de urbanização - A.4
Proporção de menores de 5 anos de idade na população - A.13
Proporção de idosos na população - A.14
Índice de envelhecimento - A.15
Razão de dependência - A.16
Razão entre nascidos vivos informados e estimados - A.17
Taxa bruta de natalidade - A.7
Taxa específica de fecundidade - A.6
Taxa de fecundidade total - A.5
Razão entre óbitos informados e estimados - A.18
Mortalidade proporcional por idade - A.8
Mortalidade proporcional por idade, em menores de 1 ano de idade - A.9
Taxa bruta de mortalidade - A.10
Esperança de vida ao nascer - A.11
Esperança de vida aos 60 anos de idade - A.12



Os objetos Tag do BeautifulSoup possuem um atributo attrs que retorna a lista com todos os atributos contidos naquele elemento html.
Veja que a tag <font color='blue'>li</font> não possui nenhum atributo associado. A tag aninhada mais abaixo: <font color='blue'>a</font> é que possui atributos do tipo <font color='blue'>href</font>.

In [30]:
print(div1.ul.li.attrs)
print("==============================================")
print(div1.ul.li.a.attrs)

{}
{'href': '/cgi/deftohtm.exe?idb2012/a01.def'}


Para finalizar, vamos criar uma lista com todos os atributos <font color='blue'>href</font> contidos em tags <font color='blue'>a</font> da <font color='blue'>div1</font>.

In [53]:
links = list()
for a in div1.find_all(lambda tag: "href" in tag.attrs.keys()):
   # print(a.attrs)
    links.append(a.attrs["href"])
links


['/cgi/deftohtm.exe?idb2012/a01.def',
 '/cgi/deftohtm.exe?idb2012/a02.def',
 'a03.htm',
 'a04.htm',
 '/cgi/deftohtm.exe?idb2012/a13.def',
 '/cgi/deftohtm.exe?idb2012/a14.def',
 '/cgi/deftohtm.exe?idb2012/a15.def',
 '/cgi/deftohtm.exe?idb2012/a16.def',
 'a17.htm',
 'a07.htm',
 'a06.htm',
 'a05.htm',
 'a18.htm',
 '/cgi/deftohtm.exe?idb2012/a08.def',
 '/cgi/deftohtm.exe?idb2012/a09.def',
 'a10.htm',
 'a11.htm',
 'a12.htm',
 'Grupo_A.xlsx',
 'Serie_Grupo_A.xlsx',
 '#topo',
 'b01.htm',
 'b0201.htm',
 'b0202.htm',
 '/cgi/deftohtm.exe?idb2012/b03.def',
 'b08.htm',
 'b09.htm',
 'b04.htm',
 'b0501.htm',
 'b0502.htm',
 'b06.htm',
 'b07.htm',
 'b10.htm',
 'Grupo_B.xlsx',
 'Serie_Grupo_B.xlsx',
 '#topo',
 'c01.htm',
 'c0101.htm',
 'c0102.htm',
 'c0104.htm',
 'c0103.htm',
 'c02.htm',
 'c16.htm',
 'c03.htm',
 '/cgi/deftohtm.exe?idb2012/c18.def',
 '/cgi/deftohtm.exe?idb2012/c04.def',
 '/cgi/deftohtm.exe?idb2012/c05.def',
 '/cgi/deftohtm.exe?idb2012/c06.def',
 '/cgi/deftohtm.exe?idb2012/c07.def',
 '/c